<a href="https://colab.research.google.com/github/Gaukhar-ai/for_my_Thinkful_work/blob/master/time_series_life%2Csales_interpolation%2Cdeletion%2Cimputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
sales = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/web_events.csv')
sales.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [3]:
sales.dtypes

timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

In [4]:
sales['timestamp'] = pd.to_datetime(sales['timestamp'], unit = 'ms')
sales.head()

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 05:02:12.117,257597,view,355908,NaN
1,2015-06-02 05:50:14.164,992329,view,248676,NaN
2,2015-06-02 05:13:19.827,111016,view,318965,NaN
3,2015-06-02 05:12:35.914,483717,view,253185,NaN
4,2015-06-02 05:02:17.106,951259,view,367447,NaN


In [5]:
sales['timestamp'].max()

Timestamp('2015-09-18 02:59:47.788000')

In [6]:
sales['timestamp']

0         2015-06-02 05:02:12.117
1         2015-06-02 05:50:14.164
2         2015-06-02 05:13:19.827
3         2015-06-02 05:12:35.914
4         2015-06-02 05:02:17.106
                    ...          
2756096   2015-08-01 03:13:05.939
2756097   2015-08-01 03:30:13.142
2756098   2015-08-01 02:57:00.527
2756099   2015-08-01 03:08:50.703
2756100   2015-08-01 03:36:03.914
Name: timestamp, Length: 2756101, dtype: datetime64[ns]

In [7]:
sales['timestamp'].min()

Timestamp('2015-05-03 03:00:04.384000')

In [8]:
sales['timestamp'] = pd.to_datetime(sales['timestamp'])
sales.dtypes

timestamp        datetime64[ns]
visitorid                 int64
event                    object
itemid                    int64
transactionid           float64
dtype: object

In [9]:
hz = sales[(sales['timestamp']> '1970-01-01 00:23:58.398785939')
          & (sales['timestamp']< '1970-01-01 00:23:53.221332117')]
print(hz['timestamp'].min())
print(hz['timestamp'].max())

NaT
NaT


In [10]:
from datetime import timedelta

In [11]:
future = sales['timestamp'] - timedelta(days=7)
future

0         2015-05-26 05:02:12.117
1         2015-05-26 05:50:14.164
2         2015-05-26 05:13:19.827
3         2015-05-26 05:12:35.914
4         2015-05-26 05:02:17.106
                    ...          
2756096   2015-07-25 03:13:05.939
2756097   2015-07-25 03:30:13.142
2756098   2015-07-25 02:57:00.527
2756099   2015-07-25 03:08:50.703
2756100   2015-07-25 03:36:03.914
Name: timestamp, Length: 2756101, dtype: datetime64[ns]

In [12]:
past = sales['timestamp'] - timedelta(days = 360)
past

0         2014-06-07 05:02:12.117
1         2014-06-07 05:50:14.164
2         2014-06-07 05:13:19.827
3         2014-06-07 05:12:35.914
4         2014-06-07 05:02:17.106
                    ...          
2756096   2014-08-06 03:13:05.939
2756097   2014-08-06 03:30:13.142
2756098   2014-08-06 02:57:00.527
2756099   2014-08-06 03:08:50.703
2756100   2014-08-06 03:36:03.914
Name: timestamp, Length: 2756101, dtype: datetime64[ns]

In [13]:
half_day = sales['timestamp'] + timedelta(hours=12)
print(half_day)

0         2015-06-02 17:02:12.117
1         2015-06-02 17:50:14.164
2         2015-06-02 17:13:19.827
3         2015-06-02 17:12:35.914
4         2015-06-02 17:02:17.106
                    ...          
2756096   2015-08-01 15:13:05.939
2756097   2015-08-01 15:30:13.142
2756098   2015-08-01 14:57:00.527
2756099   2015-08-01 15:08:50.703
2756100   2015-08-01 15:36:03.914
Name: timestamp, Length: 2756101, dtype: datetime64[ns]


In [14]:
sales['hour'] = sales['timestamp'].dt.hour
sales

,timestamp,visitorid,event,itemid,transactionid,hour
0,2015-06-02 05:02:12.117,257597,view,355908,NaN,5
1,2015-06-02 05:50:14.164,992329,view,248676,NaN,5
2,2015-06-02 05:13:19.827,111016,view,318965,NaN,5
3,2015-06-02 05:12:35.914,483717,view,253185,NaN,5
4,2015-06-02 05:02:17.106,951259,view,367447,NaN,5
...,...,...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427,NaN,3
2756097,2015-08-01 03:30:13.142,762376,view,115946,NaN,3
2756098,2015-08-01 02:57:00.527,1251746,view,78144,NaN,2
2756099,2015-08-01 03:08:50.703,1184451,view,283392,NaN,3


In [15]:
sales['timestamp'].dt.round('H')
sales

,timestamp,visitorid,event,itemid,transactionid,hour
0,2015-06-02 05:02:12.117,257597,view,355908,NaN,5
1,2015-06-02 05:50:14.164,992329,view,248676,NaN,5
2,2015-06-02 05:13:19.827,111016,view,318965,NaN,5
3,2015-06-02 05:12:35.914,483717,view,253185,NaN,5
4,2015-06-02 05:02:17.106,951259,view,367447,NaN,5
...,...,...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427,NaN,3
2756097,2015-08-01 03:30:13.142,762376,view,115946,NaN,3
2756098,2015-08-01 02:57:00.527,1251746,view,78144,NaN,2
2756099,2015-08-01 03:08:50.703,1184451,view,283392,NaN,3


In [16]:
sales['min'] = sales['timestamp'].dt.round('Min')
sales

,timestamp,visitorid,event,itemid,transactionid,hour,min
0,2015-06-02 05:02:12.117,257597,view,355908,NaN,5,2015-06-02 05:02:00
1,2015-06-02 05:50:14.164,992329,view,248676,NaN,5,2015-06-02 05:50:00
2,2015-06-02 05:13:19.827,111016,view,318965,NaN,5,2015-06-02 05:13:00
3,2015-06-02 05:12:35.914,483717,view,253185,NaN,5,2015-06-02 05:13:00
4,2015-06-02 05:02:17.106,951259,view,367447,NaN,5,2015-06-02 05:02:00
...,...,...,...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427,NaN,3,2015-08-01 03:13:00
2756097,2015-08-01 03:30:13.142,762376,view,115946,NaN,3,2015-08-01 03:30:00
2756098,2015-08-01 02:57:00.527,1251746,view,78144,NaN,2,2015-08-01 02:57:00
2756099,2015-08-01 03:08:50.703,1184451,view,283392,NaN,3,2015-08-01 03:09:00


In [17]:
life = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/life_expectancy.csv')
life.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,67.762,68.095,68.436,68.784,69.140,69.498,69.851,70.191,70.519,70.833,71.140,71.441,71.736,72.023,72.293,72.538,72.751,72.929,73.071,73.181,73.262,73.325,73.378,73.425,73.468,73.509,73.544,73.573,73.598,73.622,73.646,73.671,73.700,73.738,73.787,73.853,73.937,74.038,74.156,74.287,74.429,74.576,74.725,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.292,32.742,33.185,33.624,34.060,34.495,34.928,35.361,35.796,36.234,36.678,37.128,37.587,38.056,38.540,39.039,39.556,40.092,40.650,41.234,41.853,42.513,43.217,43.963,44.747,45.566,46.417,47.288,48.164,49.028,49.856,50.627,51.331,51.968,52.539,53.055,53.533,53.997,54.468,54.959,55.482,56.044,56.637,57.250,57.875,58.500,59.110,59.694,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,33.251,33.573,33.914,34.272,34.645,35.031,35.426,35.828,36.234,36.640,37.047,37.460,37.878,38.297,38.712,39.110,39.478,39.810,40.099,40.344,40.546,40.710,40.848,40.970,41.085,41.193,41.292,41.382,41.471,41.572,41.696,41.855,42.060,42.329,42.677,43.125,43.695,44.385,45.192,46.105,47.113,48.200,49.341,50.508,51.676,52.833,53.974,55.096,56.189,57.231,58.192,59.042,59.770,60.373,60.858,61.241,61.547
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.279,63.298,64.187,64.911,65.461,65.848,66.108,66.302,66.485,66.687,66.933,67.235,67.580,67.951,68.341,68.734,69.108,69.447,69.741,69.990,70.207,70.416,70.635,70.876,71.134,71.388,71.605,71.760,71.843,71.860,71.836,71.803,71.802,71.860,71.992,72.205,72.495,72.838,73.208,73.588,73.955,74.286,74.575,74.820,75.028,75.217,75.418,75.656,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Transform/melt the data so that the years are listed in a single column instead of separate columns.

In [19]:
ids = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
melt = set(life.columns) - set(ids)

melted = pd.melt(life, id_vars=ids,
                 value_vars=melt,
                 var_name = 'Year', value_name='Value')
melted.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,73.853
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,56.044
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,48.200
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,74.286
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,NaN


In [20]:
#Practice address missing values for countries using the different approaches (imputation, interpolation, and deletion).

In [21]:
life.isna().sum()

Country Name       0
Country Code       0
Indicator Name     0
Indicator Code     0
1960              29
                  ..
2012              17
2013              19
2014              19
2015              20
2016              19
Length: 61, dtype: int64

In [22]:
melted['Value'].fillna(melted['Value'].mean())

0        73.853000
1        56.044000
2        48.200000
3        74.286000
4        63.544406
           ...    
15043    63.544406
15044    49.630000
15045    57.526000
15046    51.442000
15047    58.828000
Name: Value, Length: 15048, dtype: float64

In [23]:
melted['Year'].fillna(melted['Year'].median())
melted.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,73.853
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,56.044
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,48.200
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,74.286
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2001,NaN


In [24]:
melted['Year'].fillna(melted['Year'].mode())

0        2001
1        2001
2        2001
3        2001
4        2001
         ... 
15043    1979
15044    1979
15045    1979
15046    1979
15047    1979
Name: Year, Length: 15048, dtype: object

In [25]:
melted['Year'].fillna(0)

0        2001
1        2001
2        2001
3        2001
4        2001
         ... 
15043    1979
15044    1979
15045    1979
15046    1979
15047    1979
Name: Year, Length: 15048, dtype: object

In [26]:
melted.isna().sum()

Country Name         0
Country Code         0
Indicator Name       0
Indicator Code       0
Year                 0
Value             1301
dtype: int64

In [27]:
melted = melted['Value'].fillna(melted['Value'].median())

In [30]:
melted.isna().sum()

0